In [ ]:
import sys

try:
    import pandas as pd
except ImportError:
    print("Instalando pandas...")
    !{sys.executable} -m pip install pandas
    import pandas as pd

try:
    import s3fs
except ImportError:
    print("Instalando s3fs...")
    !{sys.executable} -m pip install s3fs
    import s3fs

json_files = [
    's3a://bucket-raw-upa-connect-gabriel/historico_climatempo_2025.json'
]

s3_output_path = 's3a://bucket-raw-upa-connect-gabriel/clima_tempo.csv'

def process_selected_json_to_csv(json_file_list, csv_output_path):
    fs = s3fs.S3FileSystem()

    if not json_file_list:
        print("A lista de arquivos JSON está vazia.")
        return

    all_dataframes = []

    print(f"Lendo {len(json_file_list)} arquivos JSON do S3...")
    for file_path in json_file_list:
        try:
            with fs.open(file_path, 'rb') as f:
                df = pd.read_json(f)
                print(f"Colunas no arquivo {file_path}: {df.columns.tolist()}")
                all_dataframes.append(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo {file_path}: {e}")
            continue

    if not all_dataframes:
        print("Nenhum dado foi carregado com sucesso. Verifique os arquivos JSON.")
        return

    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Use 'data' se existir, senão aborta com mensagem
    date_column = 'data'
    if date_column not in combined_df.columns:
        print(f"A coluna '{date_column}' não foi encontrada. Colunas disponíveis: {combined_df.columns.tolist()}")
        return

    print(f"Convertendo '{date_column}' para datetime e ordenando os dados...")
    combined_df[date_column] = pd.to_datetime(combined_df[date_column])
    combined_df = combined_df.sort_values(by=date_column).reset_index(drop=True)

    try:
        print(f"Salvando o arquivo CSV consolidado em {csv_output_path}...")
        with fs.open(csv_output_path, 'wb') as f:
            combined_df.to_csv(f, index=False)
        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar o arquivo CSV no S3: {e}")
        print("Verifique suas permissões de escrita para o bucket S3.")

process_selected_json_to_csv(json_files, s3_output_path)

Lendo 7 arquivos JSON do S3...
Convertendo 'data_hora' e ordenando os dados...
Salvando o arquivo CSV consolidado em s3a://bucket-raw-upa-connect/tabela_historico_sensor.csv...
Processo concluído com sucesso!
